### Extract the features of the last hidden layer

In [11]:
from torchvision.models import resnet50
import torch
import numpy as np
import os
from PIL import Image
from torchvision import transforms


In [12]:
# load the pre-trained model
model = resnet50(pretrained=True)

# evaluate the model
model.eval()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
# remove the last layer (the classification layer, fc)

model = torch.nn.Sequential(*(list(model.children())[:-1]))
model.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [15]:
# apply the same transformations as the ones used during training
# https://pytorch.org/vision/0.18/models/generated/torchvision.models.resnet50.html

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

### Test with one image

In [4]:
# load the image

img = Image.open('../data/images.jpeg')

In [6]:
image_tensor = transform(img).unsqueeze(0) # add 1 more dimension to simulate a batch

image_tensor.shape # the first dimension is the batch size

torch.Size([1, 3, 224, 224])

In [7]:
# gradients are not calculated as we are using a pre-trained model
# we only want the features from the image based on the pre-trained model
with torch.no_grad():
    features = model(image_tensor)

print(features.shape)

torch.Size([1, 2048, 1, 1])


In [8]:
features = features.squeeze(0).numpy() 

In [9]:
print("Feature vector shape: ", features.shape)

Feature vector shape:  (2048, 1, 1)


### Generalize for multiple images

In [ ]:
def extract_features(image_path, model, transform):
    try:
        img = Image.open(image_path).convert('RGB') # ensure RGB format
        image_tensor = transform(img).unsqueeze(0)
        with torch.no_grad():
            features = model(image_tensor)
        return features.squeeze(0).numpy()
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

# training images
image_dir = '../data/training_images/'

all_features = []

# for each image
for image_name in os.listdir(image_dir):
    image_path = os.path.join(image_dir, image_name)
    features = extract_features(image_path, model, transform)
    if features is not None:
        all_features.append(features)

# Convert list to numpy array
all_features = np.array(all_features)
print("All features shape: ", all_features.shape)

Error processing image ../data/training_images/.DS_Store: cannot identify image file '/Users/ioannis/Library/CloudStorage/OneDrive-unipi.gr/Desktop/ioannis/university/7th semester/image analysis/assignment/data/training_images/.DS_Store'
